In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


data1 = pd.read_csv('../input/deapdata/(S01)/Preprocessed EEG Data/.csv format/S01G1AllChannels.csv')
data2 = pd.read_csv('../input/deapdata/(S01)/Preprocessed EEG Data/.csv format/S01G2AllChannels.csv')
data3 = pd.read_csv('../input/deapdata/(S01)/Preprocessed EEG Data/.csv format/S01G3AllChannels.csv')
data4 = pd.read_csv('../input/deapdata/(S01)/Preprocessed EEG Data/.csv format/S01G4AllChannels.csv')
#membual label 1
data1.insert(14, "satisfied", 2/10)
data1.insert(15, "boring ", 8/10)
data1.insert(16, "horrible ", 1/10)
data1.insert(17, "calm ", 3/10)
data1.insert(18, "funny  ",2/10)
#membual label 2
data2.insert(14, "satisfied", 7/10)
data2.insert(15, "boring ", 2/10)
data2.insert(16, "horrible ", 1/10)
data2.insert(17, "calm ", 8/10)
data2.insert(18, "funny  ",8/10)
#membual label 3
data3.insert(14, "satisfied", 2/10)
data3.insert(15, "boring ", 2/10)
data3.insert(16, "horrible ", 8/10)
data3.insert(17, "calm ", 1/10)
data3.insert(18, "funny  ",1/10)
#membual label 4
data4.insert(14, "satisfied", 8/10)
data4.insert(15, "boring ", 1/10)
data4.insert(16, "horrible ", 1/10)
data4.insert(17, "calm ", 3/10)
data4.insert(18, "funny  ",9/10)    

data1.drop(['Unnamed: 14'], axis=1, inplace=True)
data2.drop(['Unnamed: 14'], axis=1, inplace=True)
data3.drop(['Unnamed: 14'], axis=1, inplace=True) 
data4.drop(['Unnamed: 14'], axis=1, inplace=True) 

In [ ]:
data2

In [ ]:
rows = round(38252*0.5) 
cols = 14
print(rows)

In [ ]:
import numpy
from keras.models import Sequential, Input
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional
from keras.preprocessing import sequence

model = Sequential()
model.add(Input(shape=(rows,cols )))
model.add(Bidirectional(LSTM(14)))
model.add(Dense(100,activation='relu'))
model.add(Dense(5,activation='softmax'))
#model.compile(optimizer='adam', loss='mse')
model.compile(loss='BinaryCrossentropy', optimizer='adam', metrics=['accuracy'])

from keras.utils.vis_utils import plot_model
model.summary()


In [ ]:
import numpy as np
def generateX(data):
    x=data.iloc[:rows,0:cols].values
    print(x.shape)
    return x
def generateY(data):
    y=data.iloc[:rows,cols:].values[0]
    #max_i=np.max(y)
    #y=np.where(y!=np.max(y),0,1)
    print(y.shape)
    return y
def concatenate(*args):
    res=np.array(args)
    return res

In [ ]:
X=concatenate(generateX(data1),generateX(data2),generateX(data3),generateX(data4))
Y=concatenate(generateY(data1),generateY(data2),generateY(data3),generateY(data4))


In [ ]:
# print(x)
# print(y)
print(X.shape)
print(Y)

In [ ]:
model.fit(X,Y, epochs=15, verbose=2)

In [ ]:
def generateTestX(data):
    x=data.iloc[rows:,0:cols].values
    print(x.shape)
    return x
def generateTestY(data):
    y=data.iloc[rows:,cols:].values[0]
    print(y.shape)
    return y

In [ ]:
TX=generateTestX(data2)
TY=generateTestY(data2)


In [ ]:
pred=model.predict(np.array([TX]))
print(pred)
print(TY)

##### Варианты:
группы S- сессия испытания (5 сессий); G - испытуемый (по 4 на сессию);  Sensor -датчики (14 столбцов AF3...T8 на испытуемого); timestep (38252 строки на испытуемого); Classes (5 классов оценки на одного испытуемого)

**Гипотезы разбиения 70/30**

*человекоразделенное обучение*
1. разбиваем данные по сессиям 4S - тренировачная 1S - тестовая (16 / 4)
2. разбиваем данные по испытуемым 3G из каждой S - тренировачная, 1G из каждой S - тестовая (15 / 5)
*     2.1 дополнительно можно шафлить 3G тестовые блоками

*смешанное обучение*

3. Разбиваем каждую таблицу 70%/30% по строкам

*Попытка выявлять фичи по сенсору (скорее всего дурацкая идея)*

4. выделяем столбец из каждого набора сопоставляем ему классы